In [29]:
import sys
sys.path.append(r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\versatile_diffusion')

In [30]:
#pip install scikit-image

In [31]:
import os
import os.path as osp
import PIL
from PIL import Image
from pathlib import Path
import numpy as np
import numpy.random as npr

import torch
import torchvision.transforms as tvtrans
from lib.cfg_helper import model_cfg_bank
from lib.model_zoo import get_model
from lib.model_zoo.ddim_vd import DDIMSampler_VD
from lib.experiments.sd_default import color_adjust, auto_merge_imlist
from torch.utils.data import DataLoader, Dataset

from lib.model_zoo.vd import VD
from lib.cfg_holder import cfg_unique_holder as cfguh
from lib.cfg_helper import get_command_line_args, cfg_initiates, load_cfg_yaml
import matplotlib.pyplot as plt
from skimage.transform import resize, downscale_local_mean

In [35]:
import argparse
import sys

# Check if the script is being run in an interactive environment
if 'ipykernel' in sys.modules:
    # Manually set the 'sub' variable
    sub = 1  # You can change this value as needed
else:
    # Use argparse as usual
    parser = argparse.ArgumentParser(description='Argument Parser')
    parser.add_argument("-sub", "--sub", help="Subject Number", default=1, type=int)
    args = parser.parse_args()
    sub = args.sub

# Ensure that sub is within the allowed range
assert sub in [1, 2, 5, 7], "The value of 'sub' must be one of [1, 2, 5, 7]"

print(f"The value of 'sub' is: {sub}")

The value of 'sub' is: 1


In [32]:
def regularize_image(x):
        BICUBIC = PIL.Image.Resampling.BICUBIC
        if isinstance(x, str):
            x = Image.open(x).resize([512, 512], resample=BICUBIC)
            x = tvtrans.ToTensor()(x)
        elif isinstance(x, PIL.Image.Image):
            x = x.resize([512, 512], resample=BICUBIC)
            x = tvtrans.ToTensor()(x)
        elif isinstance(x, np.ndarray):
            x = PIL.Image.fromarray(x).resize([512, 512], resample=BICUBIC)
            x = tvtrans.ToTensor()(x)
        elif isinstance(x, torch.Tensor):
            pass
        else:
            assert False, 'Unknown image type'

        assert (x.shape[1]==512) & (x.shape[2]==512), \
            'Wrong image size'
        return x

In [33]:
import os
os.chdir(r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush')
print(os.getcwd())  # Verify the new working directory

C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush


In [36]:
cfgm_name = 'vd_noema'
# sampler = DDIMSampler_VD
pth = 'versatile_diffusion/pretrained/vd-four-flow-v1-0-fp16-deprecated.pth'
cfgm = model_cfg_bank()(cfgm_name)
net = get_model()(cfgm)
sd = torch.load(pth, map_location='cpu')
net.load_state_dict(sd, strict=False)


#######################
# Running in eps mode #
#######################

Load openai_unet_2d with total 859520964 parameters,99512.246 parameter sum.
Load openai_unet_0dmd with total 1708848448 parameters,249865.621 parameter sum.
Load openai_unet_vd with total 2566318852 parameters,349365.192 parameter sum.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Load pth from versatile_diffusion/pretrained/kl-f8.pth
Load autoencoderkl with total 83653863 parameters,72921.759 parameter sum.
Load optimus_bert_connector with total 109489920 parameters,19490.449 parameter sum.
Load optimus_gpt2_connector with total 132109824 parameters,18990.663 parameter sum.
Load pth from versatile_diffusion/pretrained/optimus-vae.pth
Load optimus_vae with total 241599744 parameters,-344611.688 parameter sum.
Load clip_frozen with total 427616513 parameters,64007.510 parameter sum.
Load vd 

C:\Users\sOrOush\AppData\Local\Temp\ipykernel_32344\1952740532.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(pth, map_location='cpu')


_IncompatibleKeys(missing_keys=[], unexpected_keys=['clip.model.text_model.embeddings.position_ids', 'clip.model.vision_model.embeddings.position_ids'])

In [25]:
print(f"Number of available GPUs: {torch.cuda.device_count()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")

Number of available GPUs: 1
Current CUDA device: 0


In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [27]:
# Check if CUDA is available
print("CUDA Available:", torch.cuda.is_available())

# Get the current device
if torch.cuda.is_available():
    print("CUDA Device Name:", torch.cuda.get_device_name(0))
    print("CUDA Device Count:", torch.cuda.device_count())
    print("Current CUDA Device:", torch.cuda.current_device())
else:
    print("No CUDA device detected.")

CUDA Available: True
CUDA Device Name: NVIDIA GeForce RTX 4080 Laptop GPU
CUDA Device Count: 1
Current CUDA Device: 0


In [11]:
# GPU assignments
net.clip.to(device)
net.autokl.to(device)

# If you uncomment these lines, make sure to use the correct device
# net.model.to(device)
sampler = DDIMSampler_VD(net)
#sampler.model.model.cuda(1)
#sampler.model.cuda(1)
batch_size = 1  # You can adjust this based on your GPU memory


base_path = r'C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results'
sub = 2  # replace with your subject index

# Load the predicted features from files
pred_text = np.load(os.path.join(base_path, f'predicted_features/subj{sub:02d}/nsd_cliptext_predtest_nsdgeneral.npy'))
pred_vision = np.load(os.path.join(base_path, f'predicted_features/subj{sub:02d}/nsd_clipvision_predtest_nsdgeneral.npy'))

# Try moving tensors to the GPU, and handle any errors that occur
try:
    pred_text = torch.tensor(pred_text).half().to(device)
    pred_vision = torch.tensor(pred_vision).half().to(device)
except RuntimeError as e:
    print(f"Error moving tensors to GPU: {e}")
    print("Attempting to use CPU instead...")
    device = torch.device("cpu")
    pred_text = torch.tensor(pred_text).half()
    pred_vision = torch.tensor(pred_vision).half()


In [42]:
n_samples = 1
ddim_steps = 50
ddim_eta = 0
scale = 7.5
xtype = 'image'
ctype = 'prompt'
net.autokl.half()

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0): Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (1): Module(
        (block): ModuleList(
          (0): ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (no

In [43]:
print(os.getcwd())

C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush


In [45]:
torch.manual_seed(0)
# Loop through your image generation process, without saving to disk
all_generated_images = []

for im_id in range(len(pred_vision)):
    # zim = Image.open(f'/media/SSD/04_Marco/03_fMRI_recon/generated images/VDVAE_regression_theta/{im_id}.png')
    # zim = Image.open(f'/media/SSD/04_Marco/03_fMRI_recon/generated images/VDVAE/subj{sub:02d}/{im_id}.png') # <------- # original
    # zim = Image.open(f'/media/SSD/04_Marco/03_fMRI_recon/generated images/VDVAE/negative_theta/{im_id}.png')# <-------- # negative theta
    # zim = Image.open(f'/media/SSD/04_Marco/03_fMRI_recon/generated images/VDVAE/positive_theta/{im_id}.png') # <------- positive theta
    zim = Image.open(f'C:/Users/sOrOush/SoroushProjects/14_CLIP_Ozcelic/results/generated images/VDVAE/nsdgeneral/degree1/subj01/{im_id}.png')
    zim = regularize_image(zim)
    zin = zim * 2 - 1
    zin = zin.unsqueeze(0).to(device).half()

    init_latent = net.autokl_encode(zin)

    sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)
    strength = strength_var
    assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'
    t_enc = int(strength * ddim_steps)

    z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]).to(device))

    dummy = ''
    utx = net.clip_encode_text(dummy).to(device).half()

    dummy = torch.zeros((1, 3, 224, 224)).to(device)
    uim = net.clip_encode_vision(dummy).to(device).half()

    z_enc = z_enc.to(device).half()

    h, w = 512, 512
    shape = [n_samples, 4, h // 8, w // 8]

    cim = pred_vision[im_id].unsqueeze(0).to(device)
    ctx = pred_text[im_id].unsqueeze(0).to(device)


    sampler.model.model.diffusion_model.device = device
    sampler.model.model.diffusion_model.half().to(device)
    mixing = mixing_var
    z = sampler.decode_dc(
        x_latent=z_enc,
        first_conditioning=[uim, cim],
        second_conditioning=[utx, ctx],
        t_start=t_enc,
        unconditional_guidance_scale=scale,
        xtype='image',
        first_ctype='vision',
        second_ctype='prompt',
        mixed_ratio=(1-mixing),
    )

    z = z.to(device).half()
    x = net.autokl_decode(z)
    color_adj='None'
    color_adj_flag = (color_adj.lower() != 'none') and (color_adj is not None)
    color_adj_simple = (color_adj.lower() == 'simple')
    color_adj_keep_ratio = 0.5

    if color_adj_flag and (ctype == 'vision'):
        x_adj = []
        for xi in x:
            color_adj_f = color_adjust(ref_from=(xi + 1) / 2, ref_to=color_adj_to)
            xi_adj = color_adj_f((xi + 1) / 2, keep=color_adj_keep_ratio, simple=color_adj_simple)
            x_adj.append(xi_adj)
        x = x_adj
    else:
        x = torch.clamp((x + 1.0) / 2.0, min=0.0, max=1.0)
        x = [tvtrans.ToPILImage()(xi) for xi in x]

    # Add generated image to the list
    x[0].save(f'C:/Users/sOrOush/SoroushProjects/14_CLIP_Ozcelic/results/generated_images/versatile_diffusion/nsdgeneral/degree1/subj01/{im_id}.png')
    # x[0].save(f'/media/SSD/04_Marco/03_fMRI_recon/generated images/CLIP/emonet/positve_theta/{im_id}.png')
    # x[0].save(f'/media/SSD/04_Marco/03_fMRI_recon/generated images/CLIP/emonet/negative_theta/{im_id}.png')
    # x[0].save(f'/media/SSD/04_Marco/03_fMRI_recon/generated images/CLIP/original/{im_id}.png')
    # x[0].save(f'/media/SSD/04_Marco/03_fMRI_recon/generated images/CLIP_regression_theta/{im_id}.png')


RuntimeError: Input type (torch.cuda.HalfTensor) and weight type (torch.HalfTensor) should be the same

In [16]:
import os
from PIL import Image
import torchvision.transforms as tvtrans
import torch

torch.manual_seed(0)

# Paths
input_dir = r'C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\data\generated_images_predicted_35l\subj02'
save_dir = r'C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\generated_images\versatile_diffusion\subj02\generated_clip_images_predicted_35l'
os.makedirs(save_dir, exist_ok=True)

# List all PNG files in the input folder
image_filenames = sorted([f for f in os.listdir(input_dir) if f.endswith('.png')])

for im_id, filename in enumerate(image_filenames):
    image_path = os.path.join(input_dir, filename)

    # Load and preprocess image
    zim = Image.open(image_path)
    zim = regularize_image(zim)
    zin = zim * 2 - 1
    zin = zin.unsqueeze(0).to(device).half()

    # Encode image
    init_latent = net.autokl_encode(zin)

    # Sampling schedule
    sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)
    strength = 0.75
    t_enc = int(strength * ddim_steps)

    # Add noise to latent
    z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]).to(device))

    # Dummy conditioning
    dummy_text = ''
    utx = net.clip_encode_text(dummy_text).to(device).half()

    dummy_vision = torch.zeros((1, 3, 224, 224)).to(device)
    uim = net.clip_encode_vision(dummy_vision).to(device).half()

    # Get context from pred_vision/text
    cim = pred_vision[im_id].unsqueeze(0).to(device)
    ctx = pred_text[im_id].unsqueeze(0).to(device)

    # Ensure model is on correct device
    sampler.model.model.diffusion_model.device = device
    sampler.model.model.diffusion_model.half().to(device)

    # Decode image
    mixing = 0.4
    z = sampler.decode_dc(
        x_latent=z_enc,
        first_conditioning=[uim, cim],
        second_conditioning=[utx, ctx],
        t_start=t_enc,
        unconditional_guidance_scale=scale,
        xtype='image',
        first_ctype='vision',
        second_ctype='prompt',
        mixed_ratio=(1 - mixing),
    )

    # Decode and post-process
    z = z.to(device).half()
    x = net.autokl_decode(z)

    color_adj = 'None'
    color_adj_flag = (color_adj.lower() != 'none') and (color_adj is not None)
    color_adj_simple = (color_adj.lower() == 'simple')
    color_adj_keep_ratio = 0.5

    if color_adj_flag and (ctype == 'vision'):
        x_adj = []
        for xi in x:
            color_adj_f = color_adjust(ref_from=(xi + 1) / 2, ref_to=color_adj_to)
            xi_adj = color_adj_f((xi + 1) / 2, keep=color_adj_keep_ratio, simple=color_adj_simple)
            x_adj.append(xi_adj)
        x = x_adj
    else:
        x = torch.clamp((x + 1.0) / 2.0, min=0.0, max=1.0)
        x = [tvtrans.ToPILImage()(xi) for xi in x]

    # Save image using original filename
    save_path = os.path.join(save_dir, filename)
    x[0].save(save_path)


Running DDIM Sampling with 37 timesteps


Decoding image: 100%|██████████| 37/37 [00:05<00:00,  6.34it/s]


Running DDIM Sampling with 37 timesteps


Decoding image: 100%|██████████| 37/37 [00:05<00:00,  6.44it/s]


Running DDIM Sampling with 37 timesteps


...

Running DDIM Sampling with 37 timesteps


Decoding image: 100%|██████████| 37/37 [00:05<00:00,  6.30it/s]


Running DDIM Sampling with 37 timesteps


Decoding image:   5%|▌         | 2/37 [00:00<00:08,  4.33it/s]


KeyboardInterrupt: 

In [18]:
import os
import re
from PIL import Image
import torchvision.transforms as tvtrans
import torch

torch.manual_seed(0)

# Natural sort helper for filenames
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else -1

# === Paths ===
input_dir = r'C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\data\generated_images_predicted_35l\subj02'
save_dir = r'C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\generated_images\versatile_diffusion\subj02\generated_clip_images_predicted_35l'
os.makedirs(save_dir, exist_ok=True)

# Get naturally sorted filenames
image_filenames = sorted(
    [f for f in os.listdir(input_dir) if f.endswith('.png')],
    key=extract_number
)

# === Main loop ===
for im_id in range(len(pred_vision)):

    image_path = os.path.join(input_dir, image_filenames[im_id])
    zim = Image.open(image_path)
    zim = regularize_image(zim)
    zin = zim * 2 - 1
    zin = zin.unsqueeze(0).to(device).half()

    init_latent = net.autokl_encode(zin)

    sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)
    strength = 0.75
    assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'
    t_enc = int(strength * ddim_steps)

    z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]).to(device))

    dummy = ''
    utx = net.clip_encode_text(dummy).to(device).half()

    dummy = torch.zeros((1, 3, 224, 224)).to(device)
    uim = net.clip_encode_vision(dummy).to(device).half()

    z_enc = z_enc.to(device).half()

    h, w = 512, 512
    shape = [n_samples, 4, h // 8, w // 8]

    cim = pred_vision[im_id].unsqueeze(0).to(device)
    ctx = pred_text[im_id].unsqueeze(0).to(device)

    sampler.model.model.diffusion_model.device = device
    sampler.model.model.diffusion_model.half().to(device)

    mixing = 0.4
    z = sampler.decode_dc(
        x_latent=z_enc,
        first_conditioning=[uim, cim],
        second_conditioning=[utx, ctx],
        t_start=t_enc,
        unconditional_guidance_scale=scale,
        xtype='image',
        first_ctype='vision',
        second_ctype='prompt',
        mixed_ratio=(1 - mixing),
    )

    z = z.to(device).half()
    x = net.autokl_decode(z)

    color_adj = 'None'
    color_adj_flag = (color_adj.lower() != 'none') and (color_adj is not None)
    color_adj_simple = (color_adj.lower() == 'simple')
    color_adj_keep_ratio = 0.5

    if color_adj_flag and (ctype == 'vision'):
        x_adj = []
        for xi in x:
            color_adj_f = color_adjust(ref_from=(xi + 1) / 2, ref_to=color_adj_to)
            xi_adj = color_adj_f((xi + 1) / 2, keep=color_adj_keep_ratio, simple=color_adj_simple)
            x_adj.append(xi_adj)
        x = x_adj
    else:
        x = torch.clamp((x + 1.0) / 2.0, min=0.0, max=1.0)
        x = [tvtrans.ToPILImage()(xi) for xi in x]

    # Save with same filename as input
    save_path = os.path.join(save_dir, image_filenames[im_id])
    x[0].save(save_path)


Running DDIM Sampling with 37 timesteps


Decoding image: 100%|██████████| 37/37 [00:05<00:00,  6.38it/s]


Running DDIM Sampling with 37 timesteps


Decoding image: 100%|██████████| 37/37 [00:05<00:00,  6.43it/s]


Running DDIM Sampling with 37 timesteps


...

Running DDIM Sampling with 37 timesteps


Decoding image: 100%|██████████| 37/37 [00:06<00:00,  6.04it/s]


Running DDIM Sampling with 37 timesteps


Decoding image:  86%|████████▋ | 32/37 [00:05<00:00,  5.99it/s]


KeyboardInterrupt: 